In [1]:
# import
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
# ===== 1) Read =====
file1 = "./input/AppleWatch - HeartRate StepCount etc 8440 rows - analysis.csv"
file2 = "./input/AppleWatch - HeartRate StepCount etc 92406 rows - export20200620105726.csv"

# Read CSVs; disable chunk-based dtype inference for df2 to avoid DtypeWarning.
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2, low_memory=False)

# Ensure "value" is numeric if present (coerce errors to NaN).
if "value" in df1.columns:
    df1["value"] = pd.to_numeric(df1["value"], errors="coerce")
if "value" in df2.columns:
    df2["value"] = pd.to_numeric(df2["value"], errors="coerce")

In [3]:
# ===== 2) Normalize (optional) =====
# Strip leading/trailing spaces in "type" to avoid near-duplicate names.
if "type" not in df1.columns or "type" not in df2.columns:
    raise KeyError('Both input files must contain a "type" column.')
df1["type"] = df1["type"].astype(str).str.strip()
df2["type"] = df2["type"].astype(str).str.strip()

# Helper: high-level type grouping (for easier browsing).
def type_group(t: str) -> str:
    if pd.isna(t) or not isinstance(t, str):
        return ""
    if "HKQuantityTypeIdentifier" in t:
        return "Quantity"
    if "HKCategoryTypeIdentifier" in t:
        return "Category"
    if "HKCorrelationTypeIdentifier" in t:
        return "Correlation"
    return "Other"

In [4]:
# ===== 3) Counts & summary =====
# Per-type counts for each file.
c1 = df1["type"].value_counts().rename("count_file1")
c2 = df2["type"].value_counts().rename("count_file2")

# Helper to get an example unit per type: first non-null string, or "" if none.
def first_non_null_unit_per_type(df: pd.DataFrame) -> pd.Series:
    if "unit" not in df.columns:
        # Return empty series aligned by type later
        return pd.Series(dtype="object")
    return (
        df.groupby("type")["unit"]
          .apply(lambda s: s.dropna().astype(str).iloc[0] if s.dropna().shape[0] > 0 else "")
          .rename("example_unit")
    )

u1 = first_non_null_unit_per_type(df1)
u2 = first_non_null_unit_per_type(df2)

# Union of all types across both files.
all_types = pd.Index(sorted(set(c1.index) | set(c2.index)))

# Build the summary with left joins so all types are listed ("列全").
summary = (
    pd.DataFrame(index=all_types)
      .join(c1, how="left")
      .join(c2, how="left")
)

# Fill and derive flags/aggregates.
summary["count_file1"] = summary["count_file1"].fillna(0).astype(int)
summary["count_file2"] = summary["count_file2"].fillna(0).astype(int)
summary["in_file1"] = summary["count_file1"] > 0
summary["in_file2"] = summary["count_file2"] > 0
summary["total_count"] = summary["count_file1"] + summary["count_file2"]

# Merge example units (prefer file2's example, then fallback to file1).
summary = summary.join(u2.rename("unit_example_2"), how="left")
summary = summary.join(u1.rename("unit_example_1"), how="left")
summary["unit_example"] = (
    summary["unit_example_2"]
      .where(summary["unit_example_2"].notna(), summary["unit_example_1"])
      .fillna("")
)

# Type group.
summary["type_group"] = summary.index.map(type_group)

# Sort and final column order.
summary = summary.sort_values(
    ["total_count", "type_group", "in_file2", "in_file1"],
    ascending=[False, True, False, False]
).reset_index().rename(columns={"index": "type"})

summary = summary[
    ["type_group", "type", "in_file1", "count_file1", "in_file2", "count_file2", "total_count", "unit_example"]
]

In [5]:
# ===== 4) Display (show ALL rows) =====
print(f"Unique 'type' count (union): {summary.shape[0]}")

# Display all rows in consoles / notebooks without truncation.
with pd.option_context(
    "display.max_rows", None,
    "display.max_columns", None,
    "display.width", 200,
    "display.max_colwidth", 100
):
    try:
        from IPython.display import display
        display(summary)  # show full summary (no head)
    except Exception:
        # Fallback to print if IPython is not available.
        print(summary.to_string(index=False))

Unique 'type' count (union): 18


,type_group,type,in_file1,count_file1,in_file2,count_file2,total_count,unit_example
0,Quantity,HKQuantityTypeIdentifierDistanceWalkingRunning,True,511,True,34897,35408,km
1,Quantity,HKQuantityTypeIdentifierStepCount,True,342,True,34734,35076,count
2,Quantity,HKQuantityTypeIdentifierFlightsClimbed,True,17,True,13617,13634,count
3,Quantity,HKQuantityTypeIdentifierActiveEnergyBurned,True,5028,True,5080,10108,kcal
4,Quantity,HKQuantityTypeIdentifierHeartRate,True,1320,True,1320,2640,count/min
5,Quantity,HKQuantityTypeIdentifierBasalEnergyBurned,True,974,True,974,1948,kcal
6,Quantity,HKQuantityTypeIdentifierHeadphoneAudioExposure,False,0,True,1093,1093,dBASPL
7,Quantity,HKQuantityTypeIdentifierAppleExerciseTime,True,115,True,115,230,min
8,Category,HKCategoryTypeIdentifierSleepAnalysis,False,0,True,228,228,
9,Category,HKCategoryTypeIdentifierAppleStandHour,True,104,True,104,208,


In [6]:
def _clean_and_parse_dates(df):
    # Ensure required columns exist
    for c in ["type","unit","creationDate","startDate","endDate","value"]:
        if c not in df.columns:
            df[c] = np.nan

    # Strip column values
    df["type"] = df["type"].astype(str).str.strip()

    # Parse datetimes with timezone if present
    for c in ["creationDate","startDate","endDate"]:
        df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)

    return df


def _choose_ts(df, prefer_midpoint=False):
    """
    Choose a representative timestamp per row:
    - If startDate and endDate both present and different:
        * if prefer_midpoint=True, use midpoint; else use startDate
    - Else (start==end or missing): use creationDate when available,
      otherwise fall back to whichever of startDate/endDate exists.
    """
    sd = df["startDate"]
    ed = df["endDate"]
    cd = df["creationDate"]

    has_interval = sd.notna() & ed.notna() & (sd != ed)

    if prefer_midpoint:
        midpoint = sd + (ed - sd) / 2
        ts_when_interval = midpoint
    else:
        ts_when_interval = sd

    # When interval → ts_when_interval; else → prefer creationDate, then startDate, then endDate
    ts = np.where(
        has_interval,
        ts_when_interval,
        pd.NaT
    )
    ts = pd.Series(ts, index=df.index)
    ts = ts.where(has_interval, cd).where(has_interval | cd.notna(), sd).where(has_interval | cd.notna() | sd.notna(), ed)

    df = df.copy()
    df["ts"] = ts
    return df


def gaps_summary_by_type(df, file_tag):
    """
    For each type, compute sampling-interval stats (minutes).
    Expects df with columns: ['type','unit','ts'] and ts as datetime.
    """
    use_cols = ["type", "unit", "ts"]
    d = df[use_cols].dropna(subset=["type", "ts"]).copy()

    # sort and compute per-type time gaps (minutes)
    d = d.sort_values(["type", "ts"])
    d["delta_min"] = d.groupby("type")["ts"].diff().dt.total_seconds() / 60.0

    grp = d.groupby("type")

    # compute statistics
    stats = grp["delta_min"].agg([
        ("Median gap (min)", "median"),
        ("Mean gap (min)", "mean"),
        ("P10 gap (min)", lambda s: s.quantile(0.10) if s.count() else np.nan),
        ("P90 gap (min)", lambda s: s.quantile(0.90) if s.count() else np.nan),
        ("Min gap (min)", "min"),
        ("Max gap (min)", "max"),
        ("% gaps ≤1min", lambda s: (s <= 1).mean() * 100 if s.count() else np.nan),
        ("% gaps ≤5min", lambda s: (s <= 5).mean() * 100 if s.count() else np.nan),
        ("% gaps ≤10min", lambda s: (s <= 10).mean() * 100 if s.count() else np.nan),
    ]).reset_index()

    # total samples per type (rows before diff)
    counts = grp.size().rename("Total samples").reset_index()

    # representative unit
    units = grp["unit"].agg(
        lambda x: x.dropna().astype(str).iloc[0] if x.dropna().size else ""
    ).reset_index().rename(columns={"unit": "Unit"})

    # merge
    out = (
        stats.merge(counts, on="type", how="left")
             .merge(units, on="type", how="left")
    )
    out.insert(0, "File", file_tag)

    # reorder
    cols = [
        "File","type","Unit","Total samples",
        "Median gap (min)","Mean gap (min)","P10 gap (min)","P90 gap (min)",
        "Min gap (min)","Max gap (min)",
        "% gaps ≤1min","% gaps ≤5min","% gaps ≤10min"
    ]
    out = out[cols].sort_values(["File","type"]).reset_index(drop=True)
    return out

In [7]:
# --- Prepare each file separately (no merge yet) ---
df1c = _clean_and_parse_dates(df1)
df2c = _clean_and_parse_dates(df2)

# Choose representative timestamp per your rule:
# - Use startDate when it's a real interval (start!=end)
# - Otherwise prefer creationDate
df1t = _choose_ts(df1c, prefer_midpoint=False)
df2t = _choose_ts(df2c, prefer_midpoint=False)

# Build frequency summaries
sum1 = gaps_summary_by_type(df1t, file_tag="file1")
sum2 = gaps_summary_by_type(df2t, file_tag="file2")

# Show top rows
print("File1 sampling frequency summary (per type):")
display(sum1)
print("\nFile2 sampling frequency summary (per type):")
display(sum2)

File1 sampling frequency summary (per type):


,File,type,Unit,Total samples,Median gap (min),Mean gap (min),P10 gap (min),P90 gap (min),Min gap (min),Max gap (min),% gaps ≤1min,% gaps ≤5min,% gaps ≤10min
0,file1,HKCategoryTypeIdentifierAppleStandHour,,104,60.000000,79.805825,60.000000,60.000000,60.000000,660.000000,0.000000,0.000000,0.000000
1,file1,HKCategoryTypeIdentifierHighHeartRateEvent,,3,1377.000000,1377.000000,956.293333,1797.706667,851.116667,1902.883333,0.000000,0.000000,0.000000
2,file1,HKQuantityTypeIdentifierActiveEnergyBurned,kcal,5028,1.016667,1.646476,0.516667,1.366667,0.083333,629.583333,30.489260,98.110581,98.985680
3,file1,HKQuantityTypeIdentifierAppleExerciseTime,min,115,2.100000,62.103947,1.000000,140.763333,1.000000,1235.216667,39.130435,58.260870,66.086957
4,file1,HKQuantityTypeIdentifierBasalEnergyBurned,kcal,974,0.516667,8.479325,0.500000,15.016667,0.083333,60.000000,50.924025,51.745380,55.236140
5,file1,HKQuantityTypeIdentifierDistanceWalkingRunning,km,511,1.450000,16.177320,0.515000,29.895000,0.050000,629.200000,40.313112,61.252446,67.514677
6,file1,HKQuantityTypeIdentifierFlightsClimbed,count,17,149.950000,408.300000,32.233333,896.800000,12.800000,1925.683333,0.000000,0.000000,0.000000
7,file1,HKQuantityTypeIdentifierHeartRate,count/min,1320,4.500000,6.274122,0.016667,10.073333,0.000000,628.833333,25.454545,57.272727,89.848485
8,file1,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,ms,25,240.008333,317.184722,50.143333,730.690000,12.133333,817.366667,0.000000,0.000000,0.000000
9,file1,HKQuantityTypeIdentifierStepCount,count,342,10.116667,24.194819,0.983333,42.183333,0.100000,629.200000,14.327485,39.473684,48.245614



File2 sampling frequency summary (per type):


,File,type,Unit,Total samples,Median gap (min),Mean gap (min),P10 gap (min),P90 gap (min),Min gap (min),Max gap (min),% gaps ≤1min,% gaps ≤5min,% gaps ≤10min
0,file2,HKCategoryTypeIdentifierAppleStandHour,,104,60.000000,79.805825,60.000000,60.000000,60.000000,6.600000e+02,0.000000,0.000000,0.000000
1,file2,HKCategoryTypeIdentifierHighHeartRateEvent,,3,1377.000000,1377.000000,956.293333,1797.706667,851.116667,1.902883e+03,0.000000,0.000000,0.000000
2,file2,HKCategoryTypeIdentifierMindfulSession,,3,1283.758333,1283.758333,938.338333,1629.178333,851.983333,1.715533e+03,0.000000,0.000000,0.000000
3,file2,HKCategoryTypeIdentifierSleepAnalysis,,228,45.400000,4138.065786,6.480000,1403.293333,0.200000,8.124802e+05,0.438596,5.263158,14.473684
4,file2,HKQuantityTypeIdentifierActiveEnergyBurned,kcal,5080,1.016667,201.261255,0.516667,1.366667,0.083333,1.007044e+06,30.196850,97.244094,98.228346
5,file2,HKQuantityTypeIdentifierAppleExerciseTime,min,115,2.100000,62.103947,1.000000,140.763333,1.000000,1.235217e+03,39.130435,58.260870,66.086957
6,file2,HKQuantityTypeIdentifierAppleStandTime,min,200,5.000000,41.281407,5.000000,86.000000,5.000000,1.030000e+03,0.000000,53.000000,62.000000
7,file2,HKQuantityTypeIdentifierBasalEnergyBurned,kcal,974,0.516667,8.479325,0.500000,15.016667,0.083333,6.000000e+01,50.924025,51.745380,55.236140
8,file2,HKQuantityTypeIdentifierBodyMass,kg,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,file2,HKQuantityTypeIdentifierDistanceWalkingRunning,km,34897,12.333333,56.339478,7.683333,107.750000,0.000000,2.076433e+03,1.369745,3.650744,40.496318


## Sampling Frequency Analysis of Apple Watch Data

### File1 (small file, 8,440 rows)

- **HeartRate (1,320 samples)**

  - Median gap ≈ 4.5 minutes, mean gap ≈ 6.3 minutes.
  - About 25% of gaps are ≤1 minute, 57% are ≤5 minutes, nearly 90% are ≤10 minutes.
    Indicates \~5-minute sampling at rest, switching to high frequency (≤1 minute) during workouts.

- **StepCount (342 samples)**

  - Median gap ≈ 10 minutes, mean ≈ 24 minutes, max gap >10 hours.
    Not recorded every minute, more like event-driven or sparse due to export format.

- **DistanceWalkingRunning (511 samples)**

  - Median gap ≈ 1.5 minutes, mean ≈ 16 minutes → mixture of dense and sparse intervals.
    Suggests partial continuity but many missing long stretches.

- **ActiveEnergyBurned / BasalEnergyBurned**

  - ActiveEnergyBurned: median gap ≈ 1 minute, very continuous.
  - BasalEnergyBurned: median gap ≈ 0.5 minutes, mean ≈ 8 minutes.
    Both are Apple Watch **derived metrics**, not raw sensors, updated frequently.

- **AppleStandHour (104 samples)**

  - Exactly 60-minute intervals, event-type data.

- **HeartRateVariability (SDNN, 25 samples)**

  - Median gap ≈ 240 minutes (4 hours).
    Watch measures HRV only a few times per day, very sparse.

- **HighHeartRateEvent (3 samples)**

  - Event-driven, hours or days apart.

### File2 (large file, 92k rows)

- **HeartRate (1,320 samples)**

  - Identical to File1, suggesting overlap or duplication.

- **StepCount (34,734 samples)**

  - Median gap ≈ 12 minutes, mean ≈ 57 minutes, max gap ≈ 1 day.
    Very sparse, more like cumulative step updates than continuous samples.

- **DistanceWalkingRunning (34,897 samples)**

  - Median gap ≈ 12 minutes, mean ≈ 56 minutes.
    Matches StepCount, also sparse.

- **FlightsClimbed (13,617 samples)**

  - Median gap ≈ 12.5 minutes, max gap >10 days.
    Event-driven (only recorded when stairs are climbed).

- **SleepAnalysis (228 samples)**

  - Median gap ≈ 45 minutes, mean ≈ 4,138 minutes (\~3 days), max gap ≈ 1.5 years.
    Clearly interval/event-type data, not periodic sampling.

- **RestingHeartRate / WalkingHeartRateAverage**

  - Only a handful of samples, with day-level gaps.
    Derived metrics, not continuous.

- **HeadphoneAudioExposure (1,093 samples)**

  - Median gap ≈ 10 minutes, max gap ≈ 6 days.
    Event-driven, not relevant to your analysis.

### Key Insights

1. **HeartRate**

   - The most reliable continuous data source.
   - \~5-minute sampling at rest, high-frequency (seconds) during workouts.

2. **StepCount / Distance / FlightsClimbed**

   - Sparse, event-driven logging.
   - Suitable for trend analysis but not continuous anomaly detection.

3. **Energy Burned**
   - ActiveEnergyBurned: highly continuous (\~1 min).
   - BasalEnergyBurned: frequent but **derived**, not raw sensor data.

4. **HRV / RestingHR / WalkingHR**

   - Very sparse (hours or days between samples).
   - Useful for long-term trends, not for minute-level anomaly detection.

5. **Event-type Data** (StandHour, SleepAnalysis, HighHR events)

   - Interval or event-driven, so gap statistics are less meaningful.
   - **SleepAnalysis is valuable for scene segmentation** (e.g., distinguishing sleep vs awake).